# DBSCAN

DBSCAN merupakan salah satu algoritma clustering yang mengelompokkan data berdasarkan kedekatannya dengan data lain. Data yang dianggap dekat akan dijadikan satu kelompok. Data dianggap dekat dan disebut bertetangga dengan data lainnya apabila jaraknya kurang dari smaa dengan nilai tertentu. Nilai tersebut disebut `epsilon`.

Pada DBSCAN satu *instance* data dapat dikategorikan menjadi `core_point`, `border_point`, atau `noise_data`/outlier. Sebuah data disebut `core_point` apabila memiliki jumlah tetangganya lebih dari sama dengan nilai tertentu. Nilai tersebut disebut `min_pts`. Sebuah data dikatakan `border_point` apabila jumlah tetangganya tidak lebih dari `min_pts` namun memiliki tetangga yang merupakan `core_point`. Sedangkan `noise_data` atau outlier adalah data yang jumlah tetangganya tidak lebih dari `min_pts` dan tidak bertetangga dengan `border_point`. 

Setiap `core_point` dan tetangganya (baik itu `core_point` atau pun `border_point`) akan menjadi satu cluster yang sama. `noise_data` atau outlier merupakan data yang tidak memiliki cluster. 

Ilustrasi: 
![DBSCAN](dbscan.png "Ilustrasi DBSCAN")

Pada gambar diatas, titik yang berwarna merah merupakan `core_point`, titik yang berwarna kuning merupakan `border_point` dan titik yang berwarna biru merupakan `noise_data` atau outlier.

Perhitungan jarak yang dapat digunakan pada implemetasi DBSCAN ini ada dua macam yaitu jarak euclidean dan jarak manhtattan.

Berikut ini merupakan pseudocode dari DBSCAN:

```
DBSCAN(data, eps, min_pts):
    curr_label = 0
    for data_i in data:
        if data_i is core_point and not yet labelled:
            label = curr_label
            cluster(data_i) = label
            neighbour_stack = [neighbour(data_i)]
            while neighbour_stack is not empty:
                neighbour_data_i = neighbour_stack.pop
                if neighbour_data_i not yet labelled:
                    cluster(neighbour_data_i) = label
                    if neighbour_data_i is core point:
                        neighbour_stack,push(neighbour(neighbour_data_i))
           curr_label += 1
```

In [1]:
import numpy as np 
import math

class tes_DBSCAN:
    
    UNLABELLED_DATA = -1
    
    n_clusters = None
    result = None
        
    metrics = 'euclidean'    
    eps = 0.5
    min_pts = 5
    available_metrics = ['euclidean', 'manhattan']

    def __init__ (self, min_pts=min_pts, eps=eps, metrics=metrics):
        '''
        Inisiasi kelas dengan min_pts dan epsilon
        '''
        if eps <= 0:
            raise Exception('eps must be higher than 0')
        if min_pts <= 0:
            raise Exception('min_pts must be higher than 0')
        if metrics not in self.available_metrics:
            raise Exception('No metrics \'' + str(metrics) + '\'. Available metrics '+ str(self.available_metrics))
            
        self.min_pts = min_pts
        self.eps = eps
        self.metrics=metrics
        
    def __euclidean_distance(self, point_a, point_b):
        '''
        Fungsi untuk menghitung euclidean distance
        '''
        dist = 0
        for a, b in zip(point_a, point_b):
            dist += (a - b) * (a - b)
        return np.sqrt(dist)

    def __manhattan_distance(self, point_a, point_b):
        '''
        Fungsi untuk menghitung manhattan distance 
        '''
        dist = 0
        for a, b in zip(point_a, point_b):
            dist += abs(a - b)
        return dist
    
    def __distance(self, point_a, point_b, metrics=metrics):
        '''
        Fungsi untuk mencari jarak berdasarkan metricsnya
        '''
        if len(point_a) == len(point_b):
            if metrics == 'euclidean':
                return self.__euclidean_distance(point_a, point_b)
            if metrics == 'manhattan':
                return self.__manhattan_distance(point_a, point_b)
        else:
            raise Exception("feature length doesn't same")
    
    def fit_predict(self, data):
        '''
        Fungsi untuk mengelompolkkan data
        '''
        size_data = len(data)
        
        # generate all neighbours 
        neighbours = []
        for i in range(size_data):
            neighbour_i = []
            for j in range(size_data):
                if self.__distance(data[i], data[j], self.metrics) <= self.eps:
                    neighbour_i.append(j)
            neighbours.append(neighbour_i)
        
        # initialize label
        self.result = np.full((size_data), self.UNLABELLED_DATA)
        
        # giving label to data
        curr_label = 0
        for i in range(size_data):
            # if neighbours > min_pts (data_i is core points) and not yet labelled, then give label 
            if len(neighbours[i]) >= self.min_pts and self.result[i] == self.UNLABELLED_DATA: 
                label = curr_label
                # giving label to all neighbours
                neighbours_i = [i]
                while len(neighbours_i) > 0:
                    neigh_i = neighbours_i.pop()
                    # if not yet labelled then give label to data and the neighbours
                    if self.result[neigh_i] == self.UNLABELLED_DATA:
                        self.result[neigh_i] = label
                        # if neigh_i is core point, then give label to the neighbour
                        if len(neighbours[neigh_i]) >= self.min_pts:
                            neighbours_i += neighbours[neigh_i]
                curr_label += 1
        
        n_clusters = curr_label                
        return self.result
    
    def get_n_clusters(self):
        if self.n_clusters is None:
            print("No data")
        else:
            return self.n_clusters
    
    def get_epsilon(self):
        return self.eps

    def get_metrics(self):
        return self.metrics
    
    def get_min_pts(self):
        return self.min_pts
    

## Experiments

Berikut ini merupakan hasil eksperimen implementasi DBSCAN untuk clustering data iris menggunakan *euclidean disntance*.

In [2]:
from sklearn import datasets

iris = datasets.load_iris()
data = iris.data
label = iris.target

In [40]:
from scipy import stats

def purity(cluster_pred, label):
    outlier = False
    size_data = len(cluster_pred)
    
    data_per_cluster = [[] for i in range(len(set(cluster_pred)))]
    for i,x in enumerate(cluster_pred):
        if x == -1:
            outlier = True
        data_per_cluster[x].append(label[i])

    sum = 0
    for clust in data_per_cluster:
        sum += stats.mode(clust)[1][0]
    
    if outlier:
        sum -= stats.mode(clust)[1][0]
        size_data -= len(clust)

    return sum/size_data

In [41]:
import time

dbscan = tes_DBSCAN(eps=0.5, min_pts=4, metrics='euclidean')

start = time.time()

pred = dbscan.fit_predict(data)

print("---- Time taken: {} s ----".format(time.time() - start))
print("Cluster prediction:")
print(pred)
print("Purity: {}".format(purity(pred, label)))

---- Time taken: 0.6144461631774902 s ----
Cluster prediction:
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0
  0  0  1  1  1  1  1  1  1  2  1  1  2  1  1  1  1  1  1  1 -1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 -1  1  1  1  1  1  2  1  1
  1  1  2  1  1  1  1  1  1 -1 -1  1 -1 -1  1  1  1  1  1  1  1 -1 -1  1
  1  1 -1  1  1  1  1  1  1  1  1 -1  1  1 -1 -1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1]
Purity: 0.708029197080292


## Hasil Eksperimen

Dapat dilihat bahwa DBSCAN mampu mengelompokkan data iris dan menghasilkan nilai purity sebesar 0.708 dengan nilai  `epsilon` sebesar 0.5 dan 4 `min_pts` dalam waktu 0.614 detik.